In [ ]:
import pandas as pd

import importlib

import file_loader
importlib.reload(file_loader)

from file_loader import FileLoader

# Percorso del file CSV (usa path relativo)
csv_path = "../data/input/determinazioni_amianto_2024.csv"
# Leggi il CSV con pandas
try:
    # encoding='latin1'  → Vecchi file Windows/Europa occidentale
    # encoding='cp1252'  → Variante Windows, gestisce più simboli (€ ecc.)
    # encoding='utf-8-sig' → Da usare se la prima colonna ha caratteri strani (BOM UTF-8), tipico da Excel
    # Provare diversi encoding se si vedono caratteri strani nei dati/colonne!
    loader = FileLoader(csv_path,';','utf-8-sig')
    df = loader.load()
    print("Dati caricati correttamente:")
    print(df.head())
    print("Record iniziali:", len(df))
except Exception as e:
    print(f"Errore nel caricamento: {e}")
    raise



In [ ]:
import importlib

import amianto_cleaner
importlib.reload(amianto_cleaner)

from amianto_cleaner import AmiantoCleaner

cleaner = AmiantoCleaner(df)
df_clean = cleaner.get_cleaned()
print(df_clean.head())
print("Record dopo cleaning:", len(df_clean))

In [64]:
import sqlite3

# Crea o apre il file database
conn = sqlite3.connect("../data/db/report.db")  # crea il file se non esiste

# Salva il DataFrame in una tabella chiamata, ad esempio, "amianto"
df_clean.to_sql("amianto", conn, if_exists="replace", index=False)

result = conn.execute("SELECT COUNT(*) FROM amianto").fetchone()
print("Totale record in SQLite:", result[0])

conn.close()


Totale record in SQLite: 45
